In [1]:
import numpy as np
import gym
from collections import deque
import pyprind
from time import time

## Defining Transition Probabilities

In [2]:
P_a0 = np.array([
        [0.5, 0.5],
        [0.0, 1.0]
    ])
P_a1 = np.array([
        [0.0, 1.0],
        [0.0, 1.0]
    ])
P_a2 = np.array([
        [1.0, 0.0],
        [0.0, 1.0]
    ])
P_all = np.concatenate([P_a0, P_a1, P_a2], axis=0)
P_list = [P_a0, P_a1, P_a2]

## Defining Reward Matrix

In [3]:
R_all = np.array([
        [5.0, 10.0, -1000],
        [-1000, -1000, -1.0]
    ])

## Setting discount factor $\gamma$ and tolerance Bellman Error $\tau$

In [8]:
gamma = 0.95
tau = 1e-10

## Initializing Value Function Vector

In [9]:
V = np.zeros(2)
policy = np.zeros(2, dtype=int)

## Value Iteration:

In [10]:
start = time()
for i in range(1000):
    V_old = np.copy(V)
    V = np.max(R_all + gamma*np.dot(P_all, V).reshape(R_all.shape, order='F'), axis=1)
    policy = np.argmax(R_all + gamma*np.dot(P_all, V).reshape(R_all.shape, order='F'), axis=1)
    error = np.linalg.norm(V-V_old, 2)
    if error < tau:
        print('Converged after {:} iterations with error {:}'.format(i+1, error))
        break
print('Time taken:', time() - start, 's')
print('Optimal Values:',V)
print('Optimal Policy:',policy)


Converged after 457 iterations with error 9.828401969972982e-11
Time taken: 0.021943092346191406 s
Optimal Values: [ -8.57142857 -20.        ]
Optimal Policy: [0 2]


## Sanity check by calculating Action Value function 

In [11]:
Q = -1000*np.ones((2,3))
A = [[0, 1], [2]]
start = time()
for i in range(1000):
    for s in range(2):
        for a in A[s]:
            res = R_all[s, a]
            for sprime in range(2):
                res += gamma*P_list[a][s, sprime]*np.max(Q[sprime])
            Q[s,a] = res
V = np.max(Q, axis=1)
print('Time taken:', time() - start, 's')
print('Optimal Values',V)


Time taken: 0.0550079345703125 s
Optimal Values [ -8.57142857 -20.        ]


## Policy Iteration

In [12]:
V_pi = np.zeros(2)
pi = np.zeros(2, dtype=int)
start = time()
for i in range(900000):
    pi_old = np.copy(pi)
    P_pi = np.array([P_list[pi[0]][0],P_list[pi[1]][1]])
    R_pi = np.array([R_all[0, pi[0]], R_all[1, pi[1]]])
    V_pi = np.linalg.solve(a=np.eye(len(P_pi))-gamma*P_pi, b=R_pi) #Policy Iteration
    pi = np.argmax(R_all + gamma*np.dot(P_all, V_pi).reshape(R_all.shape, order='F'), axis=1)
    if np.allclose(pi, pi_old):
        print('Converged after {:} iterations'.format(i+1))
        break
        pass
print('Time taken:', time() - start, 's')
print('Optimal Policy:',pi)
print('Optimal Values:',V_pi)

Converged after 2 iterations
Time taken: 0.0012879371643066406 s
Optimal Policy: [0 2]
Optimal Values: [ -8.57142857 -20.        ]


## Modified Policy Iteration

In [16]:
V_pi2 = np.zeros(2)
pi2 = np.zeros(2, dtype=int)
for i in range(900000):
    pi_old = np.copy(pi2)
    P_pi = np.array([P_list[pi2[0]][0],P_list[pi2[1]][1]])
    R_pi = np.array([R_all[0, pi2[0]], R_all[1, pi2[1]]])
   
    V_pi2 = R_pi + gamma*np.dot(P_pi, V_pi2) #Modified Policy Iteration
    pi2 = np.argmax(R_all + gamma*np.dot(P_all, V_pi2).reshape(R_all.shape, order='F'), axis=1)
    if np.allclose(pi2, pi_old):
        print('Converged after {:} iterations'.format(i+1))
        break
        pass
print('Time taken:', time() - start, 's')
print('Optimal Policy:',pi2)
print('Optimal Values:',V_pi2)

Converged after 2 iterations
Time taken: 81.76924300193787 s
Optimal Policy: [0 2]
Optimal Values: [-467.625 -951.   ]


## Comment on Results
   It is very difficult to draw concrete conclusions from the results of this MDP since the action and state space is very small. Discussion on the results will hence be done in the next notebook which uses a environment with a larger action and state space 
   
   

In [17]:
!which python

/Users/anilpatil/anaconda/bin/python


In [ ]:
env = gym.make("FrozenLake-v0")
#env = gym.make("Taxi-v2")

In [ ]:
N = 10000
Q = np.zeros((env.env.nS, env.env.nA))
gamma = 0.1
tol = 1e-6
bar = pyprind.ProgBar(N)
for i in range(N):
    bar.update()
    Q_old = np.copy(Q)
    for s in range(env.env.nS):
        for a in list(env.env.P[s].keys()):
            res = 0.0
            for j, sprime in enumerate([k[1] for k in env.env.P[s][a]]):
                res += env.env.P[s][a][j][0]*(env.env.P[s][a][j][2] + gamma*np.max(Q[sprime]))
            Q[s,a] = res
    error = np.linalg.norm(Q-Q_old,2)
    if error < tol:
        print('Converged after {:} iterations and error is {:}'.format(i+1,error))
        break
print(bar)
V = np.max(Q, axis=1)

In [ ]:
env.env.P[5].keys()

In [ ]:
env.env.nS

In [ ]:
env.env.nS

In [ ]:
V_policy = np.zeros(env.env.nS)
policy = np.zeros(env.env.nS, dtype=int)
 

In [ ]:
env.env.P[10]

In [ ]:

nA, nS = env.env.nA, env.env.nS


In [ ]:
T = np.zeros([nS, nA, nS])
R = np.zeros([nS, nA, nS])

In [ ]:
for s in range(nS):
    for a in range(nA):
        transitions = env.env.P[s][a]
        for p_trans,next_s,rew,done in transitions:
#             print(next_s)
            T[s,a,next_s] += p_trans
            
            R[s,a,next_s] = rew
        T[s,a,:]/=np.sum(T[s,a,:])

In [ ]:
# for s in range(nS):
#      for a in range(nA):
print(env.env.P[10][2])

In [ ]:
P_list = [np.zeros([nS,nS]) for _ in range(nA)]
R_list = [-1000*np.ones((nS, nS)) for _ in range(nA)]
for s in range(nS):
    for a in range(nA):
        transitions = env.env.P[s][a]
        for p_trans,next_s,rew,done in transitions:
            P_list[a][s,next_s]= p_trans
            R_list[a][s,next_s] = rew
       

In [ ]:
R_all = np.vstack([np.sum(R_list[i]*P_list[i], axis=1) for i in range(nA)]).T

In [ ]:
R_all

In [ ]:
T